In [2]:
import json
import os.path
import time


In [53]:
import glob
from pprint import pprint
import random
import snowballstemmer
stemmer = snowballstemmer.stemmer('swedish');

all_ids = {}
all_content = {}


for json_file in glob.glob('data/*.json'):
    data = json.load(open(json_file))
    all_ids.setdefault(data['yrkesbenamning'], []).append(data['annonsid'])
    all_content[data['annonsid']] = data

    
import json
with open('alla_annonser.json', 'w') as outfile:
    json.dump(all_content, outfile)
max_antal_jobb = 20

train_x = []
train_y = []
train_ids = []
test_x = []
test_y = []
test_ids = []
val_x = []
val_y = []
val_ids = []
#pprint(all_ids)
sorted_list = sorted(all_ids.items(), key=lambda i: len(i[1]))[-max_antal_jobb:]

for s in sorted_list:
    name = s[0]
    for aid in s[1]:
        rand = random.random()
        if rand < 0.1:
            val_y.append(name)
            val_x.append(' '.join(stemmer.stemWords(all_content[aid]['annonstext'].split())))
            val_ids.append(aid)
        elif rand < 0.2:
            test_y.append(name)
            test_x.append(' '.join(stemmer.stemWords(all_content[aid]['annonstext'].split())))
            test_ids.append(aid)
        else:
            train_y.append(name)
            train_x.append(' '.join(stemmer.stemWords(all_content[aid]['annonstext'].split())))
            train_ids.append(aid)
#pprint(sorted_list)

In [4]:
import pandas as pd
import zipfile
import io
import urllib
import json

df = pd.DataFrame()
with open('alla_annonser.json', 'r') as outfile:
    df = pd.read_json(outfile, orient='index')
print(df['annonstext'])

0017-322354    Tattys skönhetsvård ligger i Vällingby centrum...
0017-371112    Vi växer och behöver nu anställa fler vvs-mont...
0017-371140    Vi växer och behöver nu anställa fler elektrik...
0017-377779    Shs Gruppen AB söker nu snickare!\r\n\r\nArbet...
0017-380837    Verksamhetsbeskrivning:\r\n\r\nTM Markmiljö AB...
0017-408003    Verksamhetsbeskrivning:\r\n\r\nTM Markmiljö AB...
0017-415041    Du behöver vara kvinna som söker denna tjänst ...
0017-438176    Vi behöver omgående stärka upp vår bemanning p...
0017-438520    3 Servitörerer/servitriser till Indian Garden,...
0017-448937    Nu behöver vi förstärkning till vår kundmottag...
0017-452240    Verksamhetsbeskrivning\r\n\r\nVi är ett litet ...
0017-452878    Verksamhetsbeskrivning\r\nVi är ett litet före...
0017-478228    Mohlins Bussar expanderar!\r\n\r\nFöretaget sö...
0017-482670    Verksamhetsbeskrivning:\r\n\r\nBolaget skall p...
0017-482870    Verksamhetsbeskrivning:\r\n\r\nBolaget skall p...
0017-483866    Vi söker t

In [5]:
len(sample_x)

1557

In [37]:
print(sample_x[8])

Om jobbet Idag arbet mång av vår kund inom mjukvaruutveckling mot mer agil arbetssät och vi ser därför ett ökat behov av produktäg på marknaden! Sökord: Produktägare, Agile, Product owner, Mjukvaruutveckling, Software, Scrum Master, Teknisk projektledning I roll som produktäg komm du ha ett övergrip ansv för en produk und hel dess livscykel. Du komm få ta roll som spindeln i nätet och hanter olik intressenter önskemål, krav och behov. Det komm då var upp dig till att prioriter dess och håll backlog uppdaterad. På ÅF är vi stor inom produktutveckling i bransch som telecom, fordon och medtech och jobb med båd små och stor bolag. Kund finn i Stockholmsregion och uppdrag utför i viss fall på plat hos kund och i andr fall på vårt huvudkont i Solna. På ÅF tyck vi även att det är vikt att du triv på jobbet! Därför erbjud frukostseminarium, afterwork, aktivitetsklubbar, föreläsning och friskvård. Vem är du? Vi sök dig som är certifierad/utbild Scrum Mast ell agil produktäg samt har jobb som de

In [67]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree

vecto = TfidfVectorizer(max_features = 100000, max_df = 0.35, min_df = 2, sublinear_tf=True, binary = True)
#classifier = tree.DecisionTreeClassifier(max_depth = 10, max_leaf_nodes = max_antal_jobb)
classifier = SGDClassifier(alpha=0.002, epsilon=0.05, average=False)
text_clf = Pipeline([
                     ('tfidf', vecto),
                     ('clf', classifier)])
text_clf.fit(train_x, train_y)
print(classifier)

/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=0.002, average=False, class_weight=None, epsilon=0.05,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)


In [65]:
from sklearn.model_selection import GridSearchCV
parameters = {
              'tfidf__max_df':[0.35],
              'tfidf__min_df':[2],
              'clf__alpha':[0.002],
              'clf__average':[False],
              'clf__epsilon':[0.05],
              }

grid = GridSearchCV(text_clf, parameters, cv=5, verbose=100)
grid.fit(test_x, test_y)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] clf__alpha=0.002, clf__average=False, clf__epsilon=0.05, tfidf__max_df=0.35, tfidf__min_df=2 


/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


[CV]  clf__alpha=0.002, clf__average=False, clf__epsilon=0.05, tfidf__max_df=0.35, tfidf__min_df=2, score=0.7548387096774194, total=   0.3s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[CV] clf__alpha=0.002, clf__average=False, clf__epsilon=0.05, tfidf__max_df=0.35, tfidf__min_df=2 


/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


[CV]  clf__alpha=0.002, clf__average=False, clf__epsilon=0.05, tfidf__max_df=0.35, tfidf__min_df=2, score=0.7094594594594594, total=   0.3s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.0s remaining:    0.0s
[CV] clf__alpha=0.002, clf__average=False, clf__epsilon=0.05, tfidf__max_df=0.35, tfidf__min_df=2 


...

[CV]  clf__alpha=0.002, clf__average=False, clf__epsilon=0.05, tfidf__max_df=0.35, tfidf__min_df=2, score=0.7361111111111112, total=   0.3s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    2.1s remaining:    0.0s
[CV] clf__alpha=0.002, clf__average=False, clf__epsilon=0.05, tfidf__max_df=0.35, tfidf__min_df=2 


/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


[CV]  clf__alpha=0.002, clf__average=False, clf__epsilon=0.05, tfidf__max_df=0.35, tfidf__min_df=2, score=0.717391304347826, total=   0.3s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.6s finished


/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=100000, min_df=4,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__max_df': [0.35], 'tfidf__min_df': [2], 'clf__alpha': [0.002], 'clf__average': [False], 'clf__epsilon': [0.05]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=100)

In [66]:
print(grid.best_params_)

{'clf__alpha': 0.002, 'clf__average': False, 'clf__epsilon': 0.05, 'tfidf__max_df': 0.35, 'tfidf__min_df': 2}


In [68]:
from sklearn import metrics
predicted = text_clf.predict(val_x)
print(metrics.classification_report(val_y, predicted))

                                           precision    recall  f1-score   support

                          Account manager       0.72      0.79      0.76        39
                  Administrativ assistent       0.81      0.81      0.81        32
                      Barnflicka/Barnvakt       0.95      1.00      0.98        40
                              Barnskötare       0.78      0.89      0.83        35
                Butikssäljare, fackhandel       0.86      1.00      0.93        32
                         Ekonomiassistent       0.95      0.88      0.91        24
         Helpdesktekniker/Supporttekniker       1.00      0.83      0.91        42
                              Innesäljare       0.86      0.40      0.55        30
                    Kundtjänstmedarbetare       0.84      0.93      0.88        72
                            Lagerarbetare       0.90      0.87      0.88        30
          Lärare i förskola/Förskollärare       0.96      0.75      0.84        36
   

In [69]:
print(text_clf.predict(["I have a few passions in my life and one of them is doing crazy innovative apps for the mobile platform. My current app project that I do in my spare time utilizes image recognition in an unexpected way. My two other passions are golf and learning a new language - which I do whenever I have time.\r\n\r\nAnother thing I like to do is helping people, so if you think I might be able to help you in some way, don't hesitate to ask. I think this creates meaningful relationships in addition to the fact that I like to get engaged in solving problems.\r\n\r\nI've always had the goal of living and working abroad and this year, 2015, I'm moving to Tokyo.\r\n\r\n\r\nRecent accomplishments\r\n\r\nCreating and maintaining 20+ professional Android apps for both big (multi-national) and small companies while at Monterosa. Been lead Android developer for several of these apps.\r\nSome examples of apps I've been working with:\r\n- Banking\r\n- Financial trading\r\n- Online auction\r\n- Messaging\r\n- Ad campaigns (games, etc)\r\n- Social\r\n- Many more..\r\n\r\nImproved and maintained high performance financial trading software while at Orc Software.\r\n\u00B7 Scrum Master certification.\r\n\u00B7 Lead an effort to improve the performance even further.\r\n\u00B7 Worked with customers to solve (performance) problems and to improve new functionality.\r\n\r\nFinished in the top two in all of the five single-day mobile app competitions I've entered.\r\n\r\nTaken every opportunity given to work or study abroad:\r\n\u00B7 2009 - Hong Kong - Studies\r\n\u00B7 2011 - Chicago - Work\r\n\u00B7 2012 - London - Work\r\n\r\n\r\nSpecialties: Android, Java, Android Studio, Gradle, Eclipse, Inventing stuff, Algorithms, C, C++, Git, SVN, Kanban, Scrum, Linux, Optimization, TDD, XML, Objective-C"]))

['Systemutvecklare/Programmerare']


In [543]:
tree.export_graphviz(classifier, feature_names=vecto.get_feature_names(), class_names=classifier.classes_, filled=True, rounded=True,  
                         special_characters=True)

/anaconda/lib/python3.6/site-packages/sklearn/tree/export.py:386: DeprecationWarning: out_file can be set to None starting from 0.18. This will be the default in 0.20.
  DeprecationWarning)


AttributeError: 'SGDClassifier' object has no attribute 'tree_'

In [70]:
print(classifier.classes_)

['Account manager' 'Administrativ assistent' 'Barnflicka/Barnvakt'
 'Barnskötare' 'Butikssäljare, fackhandel' 'Ekonomiassistent'
 'Helpdesktekniker/Supporttekniker' 'Innesäljare' 'Kundtjänstmedarbetare'
 'Lagerarbetare' 'Lärare i förskola/Förskollärare' 'Mjukvaruutvecklare'
 'Personlig assistent' 'Projektledare, IT' 'Redovisningsekonom'
 'Sjuksköterska, grundutbildad' 'Systemutvecklare/Programmerare'
 'Telefonförsäljare' 'Undersköterska, hemtjänst och äldreboende'
 'Utesäljare']


In [76]:
from sklearn.neighbors import NearestNeighbors, KDTree
nbrs = NearestNeighbors(n_neighbors=20).fit(vecto.transform(x))

In [79]:
text = "Driven and dedicated developer with experience from both the public and private sectors. Soft spot for functional programming (especially Haskell), type theory and machine learning.\r\n\r\nHave had an interest in programming since childhood when I started with Pascal at 12 years of age, and went on to C ++ at age 16.\r\n\r\nThe present main expertise lies in C++, Objective-C and C#. With a Masters in computer science I have a well-founded understanding of advanced algorithms and also practical experience in machine learning with a focus on Support Vector Machines.\r\n\r\nPerceived as a positive and responsible person with good communication skills."
text2 = "I have a few passions in my life and one of them is doing crazy innovative apps for the mobile platform. My current app project that I do in my spare time utilizes image recognition in an unexpected way. My two other passions are golf and learning a new language - which I do whenever I have time.\r\n\r\nAnother thing I like to do is helping people, so if you think I might be able to help you in some way, don't hesitate to ask. I think this creates meaningful relationships in addition to the fact that I like to get engaged in solving problems.\r\n\r\nI've always had the goal of living and working abroad and this year, 2015, I'm moving to Tokyo.\r\n\r\n\r\nRecent accomplishments\r\n\r\nCreating and maintaining 20+ professional Android apps for both big (multi-national) and small companies while at Monterosa. Been lead Android developer for several of these apps.\r\nSome examples of apps I've been working with:\r\n- Banking\r\n- Financial trading\r\n- Online auction\r\n- Messaging\r\n- Ad campaigns (games, etc)\r\n- Social\r\n- Many more..\r\n\r\nImproved and maintained high performance financial trading software while at Orc Software.\r\n\u00B7 Scrum Master certification.\r\n\u00B7 Lead an effort to improve the performance even further.\r\n\u00B7 Worked with customers to solve (performance) problems and to improve new functionality.\r\n\r\nFinished in the top two in all of the five single-day mobile app competitions I've entered.\r\n\r\nTaken every opportunity given to work or study abroad:\r\n\u00B7 2009 - Hong Kong - Studies\r\n\u00B7 2011 - Chicago - Work\r\n\u00B7 2012 - London - Work\r\n\r\n\r\nSpecialties: Android, Java, Android Studio, Gradle, Eclipse, Inventing stuff, Algorithms, C, C++, Git, SVN, Kanban, Scrum, Linux, Optimization, TDD, XML, Objective-C"
text3 = "machine learning"
distances, indices = nbrs.kneighbors(vecto.transform([text3]))
print(indices)
for i in indices[0]:
    print(all_content[ids[i]]['annonsrubrik'] + ' - ' + all_content[ids[i]]['platsannonsUrl'])


[[4833 4577 4744 3234 4650 4762 4775 4620 4901 3843 3740 4971 4607  174
  4881 4006 3918 3588 4885 4966]]
Är du utvecklare och kan utmana oss? - https://www.arbetsformedlingen.se/Tjanster/Arbetssokande/Platsbanken/annonser/7495098
BI-Developer/ Data Scientist for respected travel company - https://www.arbetsformedlingen.se/Tjanster/Arbetssokande/Platsbanken/annonser/7544442
Juniora java-utvecklare Giesecke & Devrient - https://www.arbetsformedlingen.se/Tjanster/Arbetssokande/Platsbanken/annonser/7549325
Account Manager/säljare - https://www.arbetsformedlingen.se/Tjanster/Arbetssokande/Platsbanken/annonser/21988943
Utvecklare?    -Välkommen till oss! - https://www.arbetsformedlingen.se/Tjanster/Arbetssokande/Platsbanken/annonser/22017216
Utvecklare?    -Välkommen till oss! - https://www.arbetsformedlingen.se/Tjanster/Arbetssokande/Platsbanken/annonser/21856391
Python Developer till nystartat team i företag med mycket data - https://www.arbetsformedlingen.se/Tjanster/Arbetssokande/Platsb